In [22]:
import requests
import googleapiclient.discovery
import googleapiclient.errors
import json
from config import youtube_key


In [24]:
API_KEY = youtube_key
CATEGORY_ID = '28'  # DIY category
START_DATE = '2024-01-02T00:00:00Z'
END_DATE = '2024-01-31T23:59:59Z'
BASE_URL = 'https://www.googleapis.com/youtube/v3/search'


In [25]:
def get_video_ids(api_key, category_id, start_date, end_date):
    video_ids = []
    url = f"{BASE_URL}?part=snippet&publishedAfter={start_date}&publishedBefore={end_date}&videoCategoryId={category_id}&type=video&key={api_key}"

    while url:
        response = requests.get(url)
        data = response.json()

        for item in data.get('items', []):
            video_ids.append(item['id']['videoId'])

        # Check for the next page
        url = data.get('nextPageToken')
        if url:
            url = f"{BASE_URL}?part=snippet&publishedAfter={start_date}&publishedBefore={end_date}&videoCategoryId={category_id}&type=video&pageToken={url}&key={api_key}"

    return video_ids



In [26]:

video_ids = get_video_ids(API_KEY, CATEGORY_ID, START_DATE, END_DATE)


In [27]:
print(video_ids)



[]


In [ ]:
API_KEY = youtube_key  # Replace with your actual API key

def main(video_ids):
    # Set up the YouTube API client
    api_service_name = "youtube"
    api_version = "v3"

    # Build the YouTube API client using the API key
    youtube = googleapiclient.discovery.build(
        api_service_name, api_version, developerKey=API_KEY)
    
    # Make the API request to get video details
    request = youtube.videos().list(
        part="snippet,contentDetails,statistics,status,topicDetails",
        id=",".join(video_ids)  # Join the list of IDs into a comma-separated string
    )
    response = request.execute()

    # Pretty-print the JSON response
    print(json.dumps(response, indent=4))
    filename = 'video_dat.json'
    with open(filename, 'w') as file:
        json.dump(response, file, indent=4)

if __name__ == "__main__":
    main(video_ids)



{
    "kind": "youtube#videoListResponse",
    "etag": "YIUPVpqNjppyCWOZfL-19bLb7uk",
    "items": [],
    "pageInfo": {
        "totalResults": 0,
        "resultsPerPage": 0
    }
}


In [14]:
import googleapiclient.discovery
import json

API_KEY = youtube_key  # Replace with your actual API key

def main(video_ids):
    # Check if video_ids is valid
    if not video_ids or not all(isinstance(vid, str) for vid in video_ids):
        raise ValueError("Invalid or empty video_ids list")

    # Set up the YouTube API client
    api_service_name = "youtube"
    api_version = "v3"

    youtube = googleapiclient.discovery.build(
        api_service_name, api_version, developerKey=API_KEY
    )

    try:
        # Make the API request to get video details
        request = youtube.videos().list(
            part="snippet,contentDetails,statistics",
            id=",".join(video_ids)
        )
        response = request.execute()

        # Save the response to a file
        filename = "video_data_01.json"
        with open(filename, "w") as file:
            json.dump(response, file, indent=4)

        print(f"Data saved to {filename}")

    except googleapiclient.errors.HttpError as e:
        print(f"HTTP Error: {e}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

if __name__ == "__main__":
    main(video_ids)


HTTP Error: <HttpError 400 when requesting https://youtube.googleapis.com/youtube/v3/videos returned "The request specifies an invalid filter parameter.". Details: "[{'message': 'The request specifies an invalid filter parameter.', 'domain': 'youtube.parameter', 'reason': 'invalidFilters', 'location': 'parameters.', 'locationType': 'other'}]">
